## COVID-19 data processing with Pandas DataFrame

### Step by step to download, load, merge, clean and aggregate COVID-19 data
Let’s import the libraries we need

In [1]:
import pandas as pd
import numpy as np
import wget

In [2]:
!pip install wget

### 1. Download raw dataset

* Go to Johns Hopkins CSSE public <a href="https://github.com/CSSEGISandData/COVID-19">Github page</a> and navigate to csse_covid_19_data/css_covid_19_time_series/

* Click the global CSV dataset and click “Raw” button to get the data url.

* Then, keep the urls in a collection and download using wget.download()

In [3]:
# url of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]

[wget.download(url) for url in urls]

100% [............................................................................] 421144 / 421144

['time_series_covid19_confirmed_global.csv',
 'time_series_covid19_deaths_global.csv',
 'time_series_covid19_recovered_global.csv']

### 2. Loading dataset and extracting date list

In [4]:
confirmed_df = pd.read_csv('../data/covid-19/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('../data/covid-19/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('../data/covid-19/time_series_covid19_recovered_global.csv')

* Let’s have a quick look at data, for example, confirmed_df.head() shows 383 columns. It should be the same on deaths_df and recovered_df

In [5]:
confirmed_df.head()
#deaths_df.head()
#recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937


* By running confirmed_df.columns , deaths_df.columns and recovered_df.columns, all of them should output the same result.

In [6]:
confirmed_df.columns
#deaths_df.columns
#recovered_df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '1/25/21', '1/26/21', '1/27/21', '1/28/21', '1/29/21', '1/30/21',
       '1/31/21', '2/1/21', '2/2/21', '2/3/21'],
      dtype='object', length=383)

* Notice that columns are all date from the 4th column onwards and to get the list of dates confirmed_df.columns[4:]

In [7]:
confirmed_df.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '1/25/21', '1/26/21', '1/27/21', '1/28/21', '1/29/21', '1/30/21',
       '1/31/21', '2/1/21', '2/2/21', '2/3/21'],
      dtype='object', length=379)

### 3. Merging Confirmed, Deaths and Recovered

* Before merging, we need to use melt() to unpivot DataFrames from current wide format into long format. In other words, we are kinda transposing all date columns into values. Here are the main settings for that:
 * Use ‘Province/State’, ‘Country/Region’, ‘Lat’, ‘Long’ as identifier variables. We will later use them for merging.
 * Unpivot date columns with variable column ‘Date’ and value column ‘Confirmed’

In [8]:
dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

* Above should return new long DataFrames. All of them are ordered by Date and Country/Region because raw data was already ordered by 
  Country/Region and the date columns are already in ASC order.<br>

    Here is the example of <b>confirmed_df_long</b>

In [9]:
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
103462,NaN,Vietnam,14.058324,108.277199,2/3/21,1948
103463,NaN,West Bank and Gaza,31.952200,35.233200,2/3/21,160426
103464,NaN,Yemen,15.552727,48.516388,2/3/21,2122
103465,NaN,Zambia,-13.133897,27.849332,2/3/21,57489


* In addition, we have to remove recovered data for Canada due to mismatch issue (Canada recovered data is counted by Country-wise rather than Province/State-wise).

In [10]:
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

* After that, we use merge() to merge the 3 DataFrames one after another.

In [11]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

* Now, we should get a full table with Confirmed, Deaths and Recovered columns

In [12]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
103462,NaN,Vietnam,14.058324,108.277199,2/3/21,1948,35,1461.0
103463,NaN,West Bank and Gaza,31.952200,35.233200,2/3/21,160426,1857,150663.0
103464,NaN,Yemen,15.552727,48.516388,2/3/21,2122,615,1427.0
103465,NaN,Zambia,-13.133897,27.849332,2/3/21,57489,804,50479.0


### 4. Performing Data Cleaning

* There are 3 tasks we would like to do
 1. Converting Date from string to datetime
 2. Replacing missing value NaN
 3. Coronavirus cases reported from 3 cruise ships should be treated differently

* You probably already notice that the values in the new Date column are all string with m/dd/yy format. To convert Date values from string to datetime, let’s use DataFrame.to_datetime()

In [13]:
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
103462,NaN,Vietnam,14.058324,108.277199,2021-02-03,1948,35,1461.0
103463,NaN,West Bank and Gaza,31.952200,35.233200,2021-02-03,160426,1857,150663.0
103464,NaN,Yemen,15.552727,48.516388,2021-02-03,2122,615,1427.0
103465,NaN,Zambia,-13.133897,27.849332,2021-02-03,57489,804,50479.0


* Missing values NaN can be detected by running full_table.isna().sum()

In [14]:
full_table.isna().sum()

Province/State    71631
Country/Region        0
Lat                 379
Long                379
Date                  0
Confirmed             0
Deaths                0
Recovered          7959
dtype: int64

* We found a lot NaN in Province/State, and that makes sense as many countries only report the Country-wise data. However, there are 7,959 NaNs in Recovered and let’s replace them with 0.

In [15]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

* Apart from missing values, there are coronavirus cases reported from 3 cruise ships: Grand Princess, Diamond Princess and MS Zaandam. These data need to be extracted and treated differently due to Province/State and Country/Region mismatch over time.
* And here is how we extract the ship data.

In [16]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

* And to get rid of ship data from full_table :

In [17]:
full_table = full_table[~(ship_rows)]

### 5. Data Aggregation

* So far, all the Confirmed, Deaths, Recovered are existing data from raw CSV dataset. Let’s add an active cases column Active, which is calculated by active = confirmed — deaths — recovered .

In [18]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

<ipython-input-18-7048188b0ada>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


* And here is what full_table looks like now.

In [19]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
103462,NaN,Vietnam,14.058324,108.277199,2021-02-03,1948,35,1461.0,452.0
103463,NaN,West Bank and Gaza,31.952200,35.233200,2021-02-03,160426,1857,150663.0,7906.0
103464,NaN,Yemen,15.552727,48.516388,2021-02-03,2122,615,1427.0,80.0
103465,NaN,Zambia,-13.133897,27.849332,2021-02-03,57489,804,50479.0,6206.0


* Next, let’s aggregate data into Country/Region wise and group them by Date and Country/Region.

In [20]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

<ipython-input-20-b4242cc93c67>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


* sum() is to get the total count of ‘Confirmed’, ‘Deaths’, ‘Recovered’, ‘Active’ for the given Date and Country/Region.
* reset_index() reset the index and use the default one, which is Date and Country/Region.

* And here is what full_grouped looks like now

In [21]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
72005,2021-02-03,Vietnam,1948,35,1461.0,452.0
72006,2021-02-03,West Bank and Gaza,160426,1857,150663.0,7906.0
72007,2021-02-03,Yemen,2122,615,1427.0,80.0
72008,2021-02-03,Zambia,57489,804,50479.0,6206.0


* Now let’s add day wise New cases, New deaths and New recovered by deducting the corresponding accumulative data on the previous day.

In [22]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

<ipython-input-22-f53636865fab>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


And finally here is the full_grouped. Be aware of that this final output is Country-wise data with
 * Confirmed, Deaths, Recovered and Active are cumulative data.
 * New cases, New deaths and New Recovered are day wise data.
 * This DataFrames is ordered by Date and Country/Region.

In [23]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
72005,2021-02-03,Vietnam,1948,35,1461.0,452.0,66,0,1
72006,2021-02-03,West Bank and Gaza,160426,1857,150663.0,7906.0,470,8,540
72007,2021-02-03,Yemen,2122,615,1427.0,80.0,0,0,0
72008,2021-02-03,Zambia,57489,804,50479.0,6206.0,1256,10,783


* Finally, you can save this final data to a CSV file:

In [24]:
full_grouped.to_csv('../data/covid-19/COVID-19-time-series-clean-complete.csv')